# Jupyter Notebook for mainnet deployment



### Necessary packages 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [2]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [4]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [5]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [7]:
# Run this in case you want to import previous contract addresses
contracts_file_custom = "Community _contracts_20210602050907.pkl"
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

## Helper Contracts and Variables

In [ ]:
# Dont run this in case you have imported contracts from a pkl file 
contracts = {}
PREP_LIST = ["hxec79e9c1c882632688f8c8f9a07832bcabe8be8f","hxd3be921dfe193cd49ed7494a53743044e3376cd3",    "hx9e7509f86ea3ba5c139161d6e92a3982659e9f30","hxaad52424d4aec9dac7d9f6796da527f471269d2c"]
TIMESTAMP = 1622560500000000
contracts['sicx']="cxae6334850f13dfd8b50f8544d5acb126bb8ef82d"
contracts['staking']="cx9d829396d887f9292d8af488fab78ad24ab6b99a"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usds']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"
contracts['bandOracle'] = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
contracts["dex"] = "cx399dea56cf199b1c9e43bead0f6a284bdecfbf62"


# Deploying all contracts


In [31]:

deploy = ['addressProvider','daoFund','delegation','governance', 'lendingPool', 'lendingPoolCore','lendingPoolDataProvider','liquidationManager','ommToken', 'priceOracle','rewards' ,'snapshot','worker_token']

directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
        
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x54339ba1ff9484f450e55cf9aa087ba17797b6e69835d3809d8a831d7d94108d', 'blockHeight': 17558425, 'blockHash': '0x2ff0d149aaebc46c8420187dc53d430346edc60238c54adf129d52d748ac47bf', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx29130ff94cc48c2c5a88236060d5de0b5199ebb2', 'stepUsed': 1152335920, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1152335920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [32]:
pprint(contracts)

{'addressProvider': 'cx36dbad450be19335c9cfaf5c13b097f30b8d4b10',
 'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'oICX': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10',
 'oIUSDC': 'cxf4584895c037a3a494a0d7785fbf835cd5dc793a',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx29130ff94cc48c2c5a88236060d5de0b5199ebb2',
 'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'snapshot': 'cxbfdda64fb

## Deploying remaining contracts


In [ ]:
for item in deploy:
    if item not in contracts.keys():
        params = {}
        if item == "ommToken":
            params = {'_initialSupply':0, '_decimals': 18}
        elif item == "worker_token":
            params = {'_initialSupply':100, '_decimals': 18}
       
        deploy_transaction = DeployTransactionBuilder()\
            .from_(deployer_wallet.get_address())\
            .to(GOVERNANCE_ADDRESS)\
            .nid(NID)\
            .nonce(100)\
            .content_type("application/zip")\
            .content(gen_deploy_data_content(item))\
            .params(params)\
            .build()

        step_limit = icon_service.estimate_step(deploy_transaction) + 100000
        
        signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
        tx_hash = icon_service.send_transaction(signed_transaction)

        res = get_tx_result(tx_hash)
        print(res)
        contracts[item] = res.get('scoreAddress', '')

## Deploying otoken for USDS

In [10]:

params = {"_name":"Omm USDS Interest Token","_symbol":"oUSDS"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDS'] = res.get('scoreAddress', '')



contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x0af08d10699212dfaaf134853e515cab910c3329c70595426cb5a24d5dc1cafb', 'blockHeight': 17556919, 'blockHash': '0xfa9d82083687890943d7812038726ac7e326363616401078d3d130ca8d1a6672', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea', 'stepUsed': 1160127000, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1160127000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx57e31f23cd06c4622b634772ad97a1c1e21da513',
 'snapshot': 'cxbfdda64fbdd367d06d9b6b06d6282646479734fb',
 'worker_token': 'cx6e49a628f4e90e216f0bd07617bc762aac27eb72',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea'}

## Deploying otoken for icx

In [11]:
#deploy oICX 
params = {"_name":"Omm ICX interest Token","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xf3e0123f47ae3372bbccee1022a444013e0e88269862a9c423a423aa9f062d5c', 'blockHeight': 17556936, 'blockHash': '0x6f3ff9a01b08bf8863b52472238d451d5992452fd63c17e7cabe711b6455e602', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10', 'stepUsed': 1160125960, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1160125960, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx57e31f23cd06c4622b634772ad97a1c1e21da513',
 'snapshot': 'cxbfdda64fbdd367d06d9b6b06d6282646479734fb',
 'worker_token': 'cx6e49a628f4e90e216f0bd07617bc762aac27eb72',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea',
 'oICX': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10'}

## Deploying otoken for iusdc


In [12]:
#deploy oIUSDC
params = {"_name":"Omm IUSDC Interest Token","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xf344d27f68133047207670a5739e9ccb323bb0bb2e251099bf4d9d749efbc635', 'blockHeight': 17556962, 'blockHash': '0x73392611b0974afa349db19158d01f8794f552407f9f12e1751f385ad853c2a3', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxf4584895c037a3a494a0d7785fbf835cd5dc793a', 'stepUsed': 1160131640, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1160131640, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx57e31f23cd06c4622b634772ad97a1c1e21da513',
 'snapshot': 'cxbfdda64fbdd367d06d9b6b06d6282646479734fb',
 'worker_token': 'cx6e49a628f4e90e216f0bd07617bc762aac27eb72',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea',
 'oICX': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10',
 'oIUSDC': 'cxf4584895c037a3a494a0d7785fbf835cd5dc793a'}

## Other Contract Addresses

In [8]:
pprint(contracts)

{'addressProvider': 'cx36dbad450be19335c9cfaf5c13b097f30b8d4b10',
 'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'oICX': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10',
 'oIUSDC': 'cxf4584895c037a3a494a0d7785fbf835cd5dc793a',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx57e31f23cd06c4622b634772ad97a1c1e21da513',
 'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'snapshot': 'cxbfdda64fb

## Saving the pkl file


In [15]:
now = datetime.utcnow()
filename ="Community "+f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

saving contracts list as Community _contracts_20210602050907.pkl


## Updating contracts

In [44]:
# Updating mainnet contracts
contract_ = 'addressProvider'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDS':
    update = contracts['oUSDS']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update addressProvider cx36dbad450be19335c9cfaf5c13b097f30b8d4b10
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa3f66a095f231c01848bfbc7df319be218091b48cc0c3fb9ef298ae0b495d830', 'blockHeight': 17558700, 'blockHash': '0xb1f0bd6d4c727d1d3ffd2f31d046e35889c31bc9066ca3b77705102872fe0baa', 'txIndex': 2, 'to': 'cx36dbad450be19335c9cfaf5c13b097f30b8d4b10', 'scoreAddress': 'cx36dbad450be19335c9cfaf5c13b097f30b8d4b10', 'stepUsed': 1638474400, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1639723560, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

# Configuring SCOREs's parameters

In [33]:
settings_lendingPool = [
    {'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                        {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
                        {'contract': 'lendingPool', 'method': 'setDaoFund', 'params':{'_address': contracts['daoFund']}},
                        {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                        {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
                        {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
                        {'contract': 'lendingPool', 'method': 'setReward','params': {'_address':contracts['rewards']}},
                        {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
                        {'contract': 'lendingPool', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
                        {'contract': 'lendingPool', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}}
                       ]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setReward, with parameters {'_address': 'cx29130ff94cc48c2c5a88236060d5de0b5199ebb2'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xa8fee7f054403cdf796c3dd59bb9f97694f19b0898c44cc11f5142edc6f1ffb9', 'blockHeight': 17558454, 'blockHash': '0xfb0130a0a7a6acd1a460fe9c56558c8f64eb0123050b5b31eed306185623de4f', 'txIndex': 1, 'to': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73', 'stepUsed': 144480, 'stepPrice': 12500000000, 'cumulativeStepUsed': 144480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [11]:
settings_lendinPoolDataProvider=[{'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['usds'],'_sym':"USDS"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}}
            ]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSymbol, with parameters {'_reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_sym': 'USDS'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSymbol, with parameters {'_reserve': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', '_sym': 'ICX'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while 

In [12]:
settings_oToken =[{'contract': 'oUSDS', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oUSDS', 'method': 'setReserve', 'params':{'_address':contracts['usds']}},
            {'contract': 'oUSDS', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oUSDS', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'oUSDS', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b'} on the oUSDS contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'} on the oUSDS contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -3

In [13]:
setting_priceOracle =[{'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USDC','_rate':10*10**17}}

                      ]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling set_reference_data, with parameters {'_base': 'USDb', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_reference_data, with parameters {'_base': 'ICX', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom respo

In [34]:
setting_addressProvider =[  
    {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['usds']}},
                            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oUSDS']}},
                            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
                            {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
                            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
                            {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
                            {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
                            {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewards']}}
                         ]
for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setRewards, with parameters {'_address': 'cx29130ff94cc48c2c5a88236060d5de0b5199ebb2'} on the addressProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [15]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['usds'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '60000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '000000000000000000', 'slopeRate1': '80000000000000000

In [16]:
settings_oicx = [{'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'oICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -326

In [17]:
settings_oiusdc = [ {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                    {'contract': 'oIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}
                    ]
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cx65f639254090820361da483df233f6d0e69af9b7'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx83a129be4a270

In [19]:
pprint(contracts)

{'addressProvider': 'cx36dbad450be19335c9cfaf5c13b097f30b8d4b10',
 'daoFund': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24',
 'delegation': 'cxa99a5a8ed66cabf20998cd7a709ff64541badd6a',
 'governance': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73',
 'lendingPoolCore': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b',
 'lendingPoolDataProvider': 'cx83a129be4a2706bdf5593335d3e842ed7117678a',
 'liquidationManager': 'cx34c5cd572e020649fcd243296df6754835436bbf',
 'oICX': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10',
 'oIUSDC': 'cxf4584895c037a3a494a0d7785fbf835cd5dc793a',
 'oUSDS': 'cx4993f638f1e2829fe55c81a8c36f26f1e5b85aea',
 'ommToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00',
 'priceOracle': 'cx3bf2bf0bdbe962395969052b0999a1270a8a2444',
 'rewards': 'cx57e31f23cd06c4622b634772ad97a1c1e21da513',
 'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'snapshot': 'cxbfdda64fb

In [22]:
# feeProvider is daoFund for now
settings_liquidationManager =  [
    {'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
                                {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                                {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['daoFund']}}
                                ]
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setFeeProvider, with parameters {'_address': 'cx8d40e16d4b10d9dd715437a9903ef049dee3ab24'} on the liquidationManager contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xe618192f5a9ca2ddd83be52fe628dc1230bac46f7669d8d6ea8a54c49ff3423b', 'blockHeight': 17557715, 'blockHash': '0x7d52d1342acee4bc0e3109add8a4cc78d0613e66d8150a9fc6d135861bdb9ceb', 'txIndex': 1, 'to': 'cx34c5cd572e020649fcd243296df6754835436bbf', 'stepUsed': 150520, 'stepPrice': 12500000000, 'cumulativeStepUsed': 150520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [29]:
print(GOVERNANCE_ADDRESS)

cx0000000000000000000000000000000000000000


In [30]:
settings_delegation =[{'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
{'contract': 'delegation', 'method': 'addAllContributors','params': {'_preps':PREP_LIST}},
                     {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}}]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b'} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling addAllContributors, with parameters {'_preps': ['hxec79e9c1c882632688f8c8f9a07832bcabe8be8f', 'hxd3be921dfe193cd49ed7494a53743044e3376cd3', 'hx9e7509f86ea3ba5c139161d6e92a3982659e9f30', 'hxaad52424d4aec9dac7d9f6796da527f471269d2c']} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message

In [35]:
settings_ommToken =[{'contract': 'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':10 * 10**18}},
                    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}]
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cx29130ff94cc48c2c5a88236060d5de0b5199ebb2'} on the ommToken contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x270155230c85f1e448b5b4959679f7b1eadadc30cf1fcab18466fb225e9574fb', 'blockHeight': 17558497, 'blockHash': '0x18bb510ae25d7fb165dcae3b960e623c354efe22cded3b88758405993299a0a0', 'txIndex': 1, 'to': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'stepUsed': 143880, 'stepPrice': 12500000000, 'cumulativeStepUsed': 143880, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [25]:
settings_lendingPoolCore =[ {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                            {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                            {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                            {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
                            {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}}
                    ]
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setLendingPool, with parameters {'_address': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x1ca25976c834e66ae7106afffde7368e573ee240a40cffe53a1bd15475d1be41', 'blockHeight': 17557768, 'blockHash': '0xf4225961d168f994ee6ac76307ede7f7504080627c1094ce5ae29d4986f20768', 'txIndex': 1, 'to': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b', 'stepUsed': 150520, 'stepPrice': 12500000000, 'cumulativeStepUsed': 150520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [26]:
settings_priceOracle =[{'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':contracts['bandOracle']}},
                       {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setBandOracle, with parameters {'_address': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling toggleOraclePriceBool, with parameters {} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [ ]:
pprint(contracts)

In [27]:
settings_snapshot =[{'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPool']}},
                    {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setAdmin, with parameters {'_address': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setGovernance, with parameters {'_address': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32

In [37]:
# def setDistPercentage(self, _deposit: int, _borrow: int, _ommICX: int, _ommUSDb: int, _worker: int, _daoFund: int):
dist_params = {"_deposit": 2*10**17, "_borrow": 2*10**17, "_ommICX": 1*10**17, "_ommUSDS": 1*10**17, "_worker": 2*10**17, "_daoFund": 2*10**17}
setting_reward_percentage =[{'contract':'rewards','method':'setDistPercentage','params':dist_params}]
for sett in setting_reward_percentage:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])




Calling setDistPercentage, with parameters {'_deposit': 200000000000000000, '_borrow': 200000000000000000, '_ommICX': 100000000000000000, '_ommUSDS': 100000000000000000, '_worker': 200000000000000000, '_daoFund': 200000000000000000} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [38]:
settings_rewards=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setOmm','params': {'_address':contracts['ommToken']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}},
            {'contract': 'rewards', 'method': 'setAdmin','params': {'_address':contracts['governance']}},
            {'contract': 'rewards', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
            {'contract': 'rewards', 'method': 'setLpToken','params': {'_address':contracts['dex']}},
                  
            ]
for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmm, with parameters {'_address': 'cxd4534956f0b749be87b8128b5f728166c1417a00'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602

## Timestamp to be added 

In [39]:
settings_governance =[{'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                      {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                      {'contract': 'governance', 'method': 'setStartTimestamp','params': {'_timestamp': TIMESTAMP}},
                      ]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSnapshot, with parameters {'_address': 'cxbfdda64fbdd367d06d9b6b06d6282646479734fb'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xc2d10537fe55bea674b5e4ae57a2d015b8bff9396d7eef23c8b2cb6b75839c74', 'blockHeight': 17558580, 'blockHash': '0x888d300ffad06381a3a889860b52d68811498df3485dc250b8ca995b8f26c2ad', 'txIndex': 1, 'to': 'cx8ecee0018f4a56bce99ff680fd999cd6378525c3', 'stepUsed': 149920, 'stepPrice': 12500000000, 'cumulativeStepUsed': 149920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding USDS reserve in LendingPoolCore



In [40]:
params ={"_reserve": {"reserveAddress":contracts['usds'],"oTokenAddress":contracts['oUSDS'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xcda764cc4292f3aa2fb80910fc1b9159a692152eb07de89cfd6d660eb3f2e248', 'blockHeight': 17558606, 'blockHash': '0x20f1427219a36e8cb08afb5e666d75561b854c8ad7b96a056471c81ee5e101b9', 'txIndex': 1, 'to': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b', 'stepUsed': 277040, 'stepPrice': 12500000000, 'cumulativeStepUsed': 277040, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding ICX reserve

In [41]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc7ccd3208b36020df7e816a83874b6f85a9c9a2b320ccdc2d3b6beab439ce500', 'blockHeight': 17558614, 'blockHash': '0x10fa81a82f6c59115f5dba1610a45ab76b329f100ec033b281045b8a845bd497', 'txIndex': 1, 'to': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b', 'stepUsed': 276800, 'stepPrice': 12500000000, 'cumulativeStepUsed': 276800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding IUSDC reserve

In [42]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x5f2ae65426b066c70ca7a600d83fed7339a49caac767e6df6505613ff32e6857', 'blockHeight': 17558620, 'blockHash': '0x7302637c14b07b2d5102234faa710ef541828565b6721e2b4ac6222463417fb5', 'txIndex': 1, 'to': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b', 'stepUsed': 276600, 'stepPrice': 12500000000, 'cumulativeStepUsed': 276600, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Testing deposit

In [43]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8058f62759e4bc1a197941ea13bfd9f7a0b25cc31504ffb05cb25a25001a5950', 'blockHeight': 17558629, 'blockHash': '0x3a188a43a39201691bc80181cfe3ac7d43e55c8955ef97801366fed1a88e60e5', 'txIndex': 1, 'to': 'cx89e26eb42851fa87b2c94baa8518032f98a57f73', 'stepUsed': 3388620, 'stepPrice': 12500000000, 'cumulativeStepUsed': 3388620, 'eventLogs': [{'scoreAddress': 'cxdf072f9c1f6ae2a96d0aeac375a1f24200f1a03b', 'indexed': ['ReserveUpdated(Address,int,int,int,int)', 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', '0x0', '0x0'], 'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']}, {'scoreAddress': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx0000000000000000000000000000000000000000', 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', '0x44cb6b6403075c9d'], 'data': ['0x6d696e74']}, {'scoreAddress': 'cx99ae58473f7b0d760d6a1e2c97f3590307c03d10', 'indexed': ['Mint(Address,int)', 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'], 'data': ['0x44cb6b640307